In [5]:
import requests
import json
from newsapi import NewsApiClient

In [8]:
# getting our API keys without putting them on GitHub for you dirty theives to steal
with open('keys.json', 'r') as f:
    keys_dict = json.load(f)
newsapi_key = keys_dict["newsapi"]
summerizebot_key = keys_dict["summerizebot"]
summerize_request = "https://www.summarizebot.com/api/extract?apiKey=" + summerizebot_key + "&url={}"
print(newsapi_key, summerizebot_key)

JSONDecodeError: Expecting ',' delimiter: line 2 column 50 (char 51)

In [7]:
# input: article search query as a string
# does: aggregates articles based on query, then stores those articles as JSON files
def getArticles( query ):
    # get the articles with  News API
    client = NewsApiClient(api_key=newsapi_key)
    articles = client.get_everything(q=query,
                                      from_param='2019-09-20',
                                      to='2019-10-15',
                                      language='en',
                                      sort_by='relevancy',
                                      page=2)
    # for each article, make sure we have the JSON
    for article in articles["articles"]:
        # if !weAlreadyHave( article["url"]) do
        articleData = requests.get(summerize_request.format(article["url"]))
        filename = 'articles/' + articleData.json()['article title'] + '.json'
        with open(filename, 'w') as fp:
            json.dump(articleData, fp)
    return 0;